In [2]:
import pandas as pd

card = pd.read_csv('/content/card_data.csv')

In [3]:
card.head()

,Card Name,Annual_fee,Base Record,Category_List
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,['항공마일리지']
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"['쇼핑', '레저', '카페/베이커리', '편의점', '대형마트', '의료', '..."
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"['쇼핑', '주유', '카페/베이커리', '편의점', '대중교통', '간편결제',..."
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"['대중교통', '간편결제', '편의점', '통신', '카페/베이커리', '대형마트..."
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"['포인트/캐시백', '카페/베이커리', '쇼핑', '통신', '대중교통', '영화..."


### 1. Feature Vectorization

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# 리스트 데이터를 문자열로 변환
card['Category_String'] = card['Category_List'].apply(lambda x: "".join(x))

# CountVectorizer 설정
vectorizer = CountVectorizer()

# 문자열 데이터를 벡터화
X = vectorizer.fit_transform(card['Category_String'])

# 결과 출력
print("Feature names:", vectorizer.get_feature_names_out())
print("Vectorized data:\n", X.toarray())

Feature names: ['간편결제' '관리비' '교육' '납부' '대중교통' '대형마트' '레저' '문화' '반려동물' '베이커리' '뷰티' '쇼핑'
 '영화' '외식' '육아' '의료' '주유' '카페' '캐시백' '통신' '편의점' '포인트' '하이패스' '항공마일리지' '혜택']
Vectorized data:
 [[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [8]:
count_vect = CountVectorizer(ngram_range=(1,2))
category_matrix = count_vect.fit_transform(card['Category_String'])
category_matrix

<166x265 sparse matrix of type '<class 'numpy.int64'>'
	with 2004 stored elements in Compressed Sparse Row format>

In [9]:
category_matrix.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### 2. 코사인 유사도

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

cate_sim = cosine_similarity(category_matrix, category_matrix)
cate_sim.shape

(166, 166)

In [13]:
cate_sim[0, :20]

array([1.        , 0.        , 0.16439899, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [17]:
card_sim_sorted_index = cate_sim.argsort()[:, ::-1]
card_sim_sorted_index[0:10]

array([[  0,  25, 125, ..., 100,  99,  82],
       [  1,  90,  12, ..., 108, 116,   0],
       [  2,  40,  51, ...,  19, 116,  82],
       ...,
       [  7,  62, 149, ..., 116, 118,   0],
       [148,   8, 114, ..., 110, 112,   0],
       [  9,  81,  45, ..., 119, 120,   0]])

In [15]:
def find_sim_card(df, sorted_index, card_name, top_n =2):
  # 1. card name 찾기
  card_name = card[card['Card Name']==card_name]

  # 2. 찾고자 하는 카드의 인덱스 찾기
  card_index = card_name.index.values
  similar_indexes = sorted_index[card_index, :top_n]

  similar_indexes = similar_indexes.reshape(-1) # 1차원 배열로 만들어 주기

  return df.iloc[similar_indexes]


In [20]:
similar_card_df = find_sim_card(card, card_sim_sorted_index,"DA카드의정석Ⅱ")

In [21]:
similar_card_df

,Card Name,Annual_fee,Base Record,Category_List,Category_String
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"['쇼핑', '레저', '카페/베이커리', '편의점', '대형마트', '의료', '...","['쇼핑', '레저', '카페/베이커리', '편의점', '대형마트', '의료', '..."
90,아메리칸 익스프레스 로즈골드 KB국민카드,"국내 1만9천원, 해외 1만9천원",직전 1개월 합계 30만원 이상,"['쇼핑', '카페/베이커리', '편의점', '대형마트', '레저', '외식', '...","['쇼핑', '카페/베이커리', '편의점', '대형마트', '레저', '외식', '..."
